In [0]:
!git clone https://github.com/workpiles/CUCUMBER-9.git

fatal: destination path 'CUCUMBER-9' already exists and is not an empty directory.


In [0]:
#ファイルを解凍する。
!tar -zxvf ./CUCUMBER-9/prototype_1/cucumber-9-python.tar.gz

batches.meta
data_batch_5
data_batch_4
data_batch_3
data_batch_2
data_batch_1
test_batch


In [0]:
!mkdir weights

mkdir: cannot create directory ‘weights’: File exists


In [0]:
import os, math, cv2
import numpy as np
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, BatchNormalization, LeakyReLU, Reshape
from keras.layers import Conv2DTranspose, MaxPooling2D, LeakyReLU, Flatten
from keras.layers.core import Activation
from keras.utils. generic_utils import Progbar
from keras.optimizers import Adam
from pathlib import Path
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [0]:
class Generator(object):
    def __init__(self, input_dim, image_shape):
        INITIAL_CHANNELS = 128
        INITIAL_SIZE = 8

        inputs = Input((input_dim,))
        fc1 = Dense(input_dim=input_dim, units=INITIAL_CHANNELS * INITIAL_SIZE * INITIAL_SIZE)(inputs)
        fc1 = BatchNormalization()(fc1)
        fc1 = LeakyReLU(0.2)(fc1)
        fc2 = Reshape((INITIAL_SIZE, INITIAL_SIZE, INITIAL_CHANNELS), 
                      input_shape=(INITIAL_CHANNELS * INITIAL_SIZE * INITIAL_SIZE,))(fc1)
        up1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(fc2)
        conv1 = Conv2D(64, (3, 3), padding='same')(up1)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        up2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv1)
        conv2 = Conv2D(image_shape[2], (5, 5), padding='same')(up2)
        outputs = Activation('tanh')(conv2)

        self.model = Model(inputs=[inputs], outputs=[outputs])

    def get_model(self):
        return self.model

In [0]:
class Discriminator(object):
    def __init__(self, input_shape):
        inputs = Input(input_shape)
        conv1 = Conv2D(64, (5, 5), padding='same')(inputs)
        conv1 = LeakyReLU(0.2)(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, (5, 5), padding='same')(pool1)
        conv2 = LeakyReLU(0.2)(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        fc1 = Flatten()(pool2)
        fc1 = Dense(1)(fc1)
        outputs = Activation('sigmoid')(fc1)

        self.model = Model(inputs=[inputs], outputs=[outputs])

    def get_model(self):
        return self.model

In [0]:
class DCGAN(object):
    def __init__(self, input_dim, image_shape):
        self.input_dim = input_dim
        self.d = Discriminator(image_shape).get_model()
        self.g = Generator(input_dim, image_shape).get_model()

    def compile(self, g_optim, d_optim):
        self.d.trainable = False
        self.dcgan = Sequential([self.g, self.d])
        self.dcgan.compile(loss='binary_crossentropy', optimizer=g_optim)
        self.d.trainable = True
        self.d.compile(loss='binary_crossentropy', optimizer=d_optim)

    def train(self, epochs, batch_size, X_train):
        g_losses = []
        d_losses = []
        for epoch in range(epochs):
            np.random.shuffle(X_train)
            n_iter = X_train.shape[0] // batch_size
            progress_bar = Progbar(target=n_iter)
            for index in range(n_iter):
                # ランダムノイズを生成 -> N 次元の潜在ベクトル
                noise = np.random.uniform(-1, 1, size=(batch_size, self.input_dim))

                # 実データをロードして偽のデータを生成する
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                for i in range(batch_size):
                    if np.random.random() > 0.5:
                        image_batch[i] = np.fliplr(image_batch[i])
                    if np.random.random() > 0.5:
                        image_batch[i] = np.flipud(image_batch[i])
                generated_images = self.g.predict(noise, verbose=0)

                # discriminator のためのラベル付与
                X = np.concatenate((image_batch, generated_images))
                y = np.array([1] * batch_size + [0] * batch_size)

                # discriminatorの訓練
                d_loss = self.d.train_on_batch(X, y)

                # generatorの訓練
                g_loss = self.dcgan.train_on_batch(noise, np.array([1] * batch_size))

                progress_bar.update(index, values=[('g', g_loss), ('d', d_loss)])
            g_losses.append(g_loss)
            d_losses.append(d_loss)
            if (epoch+1)%10 == 0:
                image = self.combine_images(generated_images)
                image = (image + 1) / 2.0 * 255.0
                cv2.imwrite('./result/' + str(epoch) + ".png", image)
            print('\nEpoch' + str(epoch) + " end")

            # 各エポックごとに重みを保存
            if (epoch+1)%50 == 0:
                self.g.save_weights('weights/generator_' + str(epoch) + '.h5', True)
                self.d.save_weights('weights/discriminator_' + str(epoch) + '.h5', True)
        return g_losses, d_losses

    def load_weights(self, g_weight, d_weight):
        self.g.load_weights(g_weight)
        self.d.load_weights(d_weight)

    def combine_images(self, generated_images):
        num = generated_images.shape[0]
        width = int(math.sqrt(num))
        height = int(math.ceil(float(num) / width))
        shape = generated_images.shape[1:4]
        image = np.zeros((height * shape[0], width * shape[1], shape[2]),
                         dtype=generated_images.dtype)
        for index, img in enumerate(generated_images):
            i = int(index / width)
            j = index % width
            image[i * shape[0]:(i + 1) * shape[0], j * shape[1]:(j + 1) * shape[1], :] = img[:, :, :]
        return image

In [0]:
def load_image_and_label(pickled_files):
    # 各ファイルには495 画像が含まれている
    IMAGE_COUNT_PER_FILE = 495
    # 画像の形状：32x32x3
    ROW = 32
    COL = 32
    DIM = 3
    whole_images = np.empty((IMAGE_COUNT_PER_FILE*len(pickled_files), ROW, COL, DIM))
    whole_labels = np.empty(IMAGE_COUNT_PER_FILE*len(pickled_files))
    for i, pickled_file in enumerate(pickled_files):
        dict = _unpickle(pickled_file)
        images = dict['data'].reshape(IMAGE_COUNT_PER_FILE, DIM, ROW, COL).transpose(0, 2, 3, 1)
        whole_images[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE, :, :, :] = images
        labels = dict['labels']
        whole_labels[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE] = labels
    return (whole_images, whole_labels)

In [0]:
def _unpickle(pickled_file):
    import pickle

    with open(pickled_file, 'rb') as file:
        dict = pickle.load(file, encoding='latin1')
    return dict

In [0]:
def load_cucumber():
    (X1, y1) = load_image_and_label(['data_batch_1',
                                     'data_batch_2',
                                     'data_batch_3',
                                     'data_batch_4',
                                     'data_batch_5'])
    (X2, y2) = load_image_and_label(['test_batch'])
    X = np.concatenate((X1, X2), axis=0)
    y = np.concatenate((y1, y2), axis=0)
    # 2L を正常とする
    normal_index = np.where(y == 0)
    # C を異常とする
    anomaly_index = np.where(y == 8)
    X_normal = X[normal_index]
    X_anomaly = X[anomaly_index]
    y_normal = y[normal_index]
    y_anomaly = y[anomaly_index]
    # 訓練データとテストデータに分割
    X_train, X_test, y_train, y_test = train_test_split(X_normal, y_normal, 
                                                        test_size=0.2, stratify=y_normal, random_state=0)
    X_test = np.concatenate((X_test, X_anomaly), axis=0)
    y_test = np.concatenate((y_test, y_anomaly), axis=0)
    y_test = y_test == 8
    return X_train, X_test, y_test

In [0]:
def normalize(X):
    return (X-127.5)/127.5

In [0]:
def denormalize(X):
    return ((X + 1.0)/2.0*255.0).astype(dtype=np.uint8)

In [0]:
batch_size = 16
epochs = 4000
input_dim = 30
g_optim = Adam(lr=0.0001, beta_1=0.5, beta_2=0.9)
d_optim = Adam(lr=0.0001, beta_1=0.5, beta_2=0.9)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
### データの準備
X_train, X_test, y_test = load_cucumber()
X_train = normalize(X_train)
X_test = normalize(X_test)
input_shape = X_train[0].shape
X_test_original = X_test.copy()


In [0]:
### 1. generator と discriminatorの訓練
dcgan = DCGAN(input_dim, input_shape)
dcgan.compile(g_optim, d_optim)
g_losses, d_losses = dcgan.train(epochs, batch_size, X_train)
with open('loss.csv', 'w') as f:
    for g_loss, d_loss in zip(g_losses, d_losses):
        f.write(str(g_loss) + ',' + str(d_loss) + '\n')

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


13/16 [=======================>......] - ETA: 0s - g: 1.2233 - d: 0.5237
Epoch0 end
12/16 [=====================>........] - ETA: 0s - g: 1.9062 - d: 0.2543
Epoch1 end
12/16 [=====================>........] - ETA: 0s - g: 2.4582 - d: 0.1221
Epoch2 end
12/16 [=====================>........] - ETA: 0s - g: 2.6423 - d: 0.0676
Epoch3 end
12/16 [=====================>........] - ETA: 0s - g: 3.2034 - d: 0.0472
Epoch4 end
12/16 [=====================>........] - ETA: 0s - g: 3.2824 - d: 0.0506
Epoch5 end
12/16 [=====================>........] - ETA: 0s - g: 2.8918 - d: 0.0273
Epoch6 end
12/16 [=====================>........] - ETA: 0s - g: 2.4370 - d: 0.0761
Epoch7 end
12/16 [=====================>........] - ETA: 0s - g: 3.6164 - d: 0.0156
Epoch8 end
12/16 [=====================>........] - ETA: 0s - g: 2.2752 - d: 0.1866
Epoch9 end
12/16 [=====================>........] - ETA: 0s - g: 1.9714 - d: 0.1156
Epoch10 end
12/16 [=====================>........] - ETA: 0s - g: 3.0980 - d: 0.0492
Ep

### 推測

In [0]:
def sum_of_residual(y_true, y_pred):
    return K.sum(K.abs(y_true - y_pred))

In [0]:
class ANOGAN(object):
    def __init__(self, input_dim, g):
        self.input_dim = input_dim
        self.g = g
        g.trainable = False
        # 入力層は訓練できない。同じサイズと同じ分布で新しいレイヤーを追加する。
        anogan_in = Input(shape=(input_dim,))
        g_in = Dense((input_dim), activation='tanh', trainable=True)(anogan_in)
        g_out = g(g_in)
        self.model = Model(inputs=anogan_in, outputs=g_out)
        self.model_weight = None

    def compile(self, optim):
        self.model.compile(loss=sum_of_residual, optimizer=optim)
        K.set_learning_phase(0)

    def compute_anomaly_score(self, x, iterations=300):
        z = np.random.uniform(-1, 1, size=(1, self.input_dim))

        # 潜伏を変えるための学習
        loss = self.model.fit(z, x, batch_size=1, epochs=iterations, verbose=0)
        loss = loss.history['loss'][-1]
        similar_data = self.model.predict_on_batch(z)

        return loss, similar_data

In [0]:
iterations = 10
input_dim = 30
anogan_optim = Adam(lr=0.001, amsgrad=True)

In [0]:
### データの準備
X_train, X_test, y_test = load_cucumber()
X_train = normalize(X_train)
X_test = normalize(X_test)
input_shape = X_train[0].shape

In [0]:
### ANOGANの訓練
dcgan = DCGAN(input_dim, input_shape)
dcgan.load_weights('weights/generator_3999.h5', 'weights/discriminator_3999.h5')

for i, test_img in enumerate(X_test[:50]):
    test_img = test_img[np.newaxis,:,:,:]
    anogan = ANOGAN(input_dim, dcgan.g)
    anogan.compile(anogan_optim)
    anomaly_score, generated_img = anogan.compute_anomaly_score(test_img, iterations)
    generated_img = denormalize(generated_img)
    imgs = np.concatenate((denormalize(test_img[0]), generated_img[0]), axis=1)
    cv2.imwrite('predict' + os.sep + str(int(anomaly_score)) + '_' + str(i) + '.png', imgs)
    print(str(i) + ' %.2f'%anomaly_score)
    with open('scores.txt', 'a') as f:
        f.write(str(anomaly_score) + '\n')